# How to Use: `calc_mandelbrot_channel`


## Log into OpenBB

In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)


## Collect Data

In [16]:
from humbldata.toolbox.technical.mandelbrot_channel.helpers import add_window_index
from humbldata.toolbox.toolbox_helpers import log_returns

symbols = ["AAPL", "PCT", "NVDA", "SMCI", "TSLA", "AMD", "SPY", "GLD", "BTC-USD", "ETH-USD"]
rr_list = ["MSFT", "AAPL", "AMZN", "META", "GOOGL", "NFLX", "TSLA", "NVDA"]
sector_symbols = [
    "XLF",
    "XLU",
    "XLK",
    "XLE",
    "XLI",
    "XLV",
    "XLY",
    "XLP",
    "XLB",
    "XLRE",
    "XTL"
]


data = (
    obb.equity.price.historical(
        ["AAPL","GOOGL"],
        provider="yfinance",
        start_date="1950-01-01",
        end_date="2024-03-08",
        adjusted=True,
    ).to_polars()
).drop(["dividends", "stock_splits"])

data = log_returns(data, _column_name="close")


In [17]:
from humbldata.toolbox.technical.volatility.realized_volatility_model import calc_realized_volatility

data_rv = calc_realized_volatility(
    data=data,
    window="1m",
    method="std",
    grouped_mean=None,
)


## Calculate Mandelbrot Channel


In [29]:
from humbldata.toolbox.technical.mandelbrot_channel.model import calc_mandelbrot_channel

%time
mandelbrot = calc_mandelbrot_channel(
    data,
    window="3m",
    rv_adjustment=False,
    _rv_method="yz",
    _rv_grouped_mean=False,
    _rs_method="RS_min",
    _live_price=False
).collect()


CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 13.8 µs


In [30]:
mandelbrot

symbol,bottom_price,recent_price,top_price
str,f64,f64,f64
"""AAPL""",161.8013,170.73,185.5065
"""GOOGL""",45.5527,135.41,515.9767


In [6]:
mandelbrot_og = mandelbrot